### FINA 4380 with Marius Popescu

### Characteristic-Based Portfolio Formation

In [1]:
import numpy as np
import pandas as pd

import datetime as dt
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
data = pd.read_csv('month_rets.csv',
                    index_col = 'date',
                    usecols = ['date','TICKER', 'PRC', 'RET'],
                    parse_dates = True)
data.head()

,TICKER,PRC,RET
date,,,
2013-01-31,MSFT,27.450,0.027717
2013-02-28,MSFT,27.800,0.021129
2013-03-28,MSFT,28.605,0.028957
2013-04-30,MSFT,33.100,0.157140
2013-05-31,MSFT,34.900,0.061329


In [3]:
data.index=data.index+pd.offsets.MonthEnd(0)

In [4]:
#data.info()

In [5]:
data['TICKER'].unique()

array(['MSFT', 'IBM', 'FB', 'AAPL', 'AMZN', 'GOOGL'], dtype=object)

In [6]:
prices_201301 = data[['TICKER','PRC']].loc['2013-01']
prices_201301

,TICKER,PRC
date,,
2013-01-31,MSFT,27.45000
2013-01-31,IBM,203.07001
2013-01-31,FB,30.98100
2013-01-31,AAPL,455.48999
2013-01-31,AMZN,265.50000
2013-01-31,GOOGL,755.69000


In [7]:
prices_201301['Rank']=pd.qcut(prices_201301['PRC'],3,labels=['T1','T2','T3'])
#prices_201301['Rank']=pd.qcut(prices_201301['PRC'],3,labels=range(1,4))

In [8]:
prices_201301.sort_values(['PRC'])

,TICKER,PRC,Rank
date,,,
2013-01-31,MSFT,27.45000,T1
2013-01-31,FB,30.98100,T1
2013-01-31,IBM,203.07001,T2
2013-01-31,AMZN,265.50000,T2
2013-01-31,AAPL,455.48999,T3
2013-01-31,GOOGL,755.69000,T3


In [9]:
prices = data[['TICKER','PRC']]
prices.head()

,TICKER,PRC
date,,
2013-01-31,MSFT,27.450
2013-02-28,MSFT,27.800
2013-03-31,MSFT,28.605
2013-04-30,MSFT,33.100
2013-05-31,MSFT,34.900


In [10]:
prices_terciles = prices[prices.index.month == 6].copy()
prices_terciles['Rank']=prices_terciles.groupby('date')['PRC'].transform(lambda x : pd.qcut(x,3,labels=['T1','T2','T3']))
#prices['Rank']=prices.groupby('date')['PRC'].transform(lambda x : pd.qcut(x,3,labels=range(1,4)))

In [11]:
prices_terciles.reset_index().set_index(['date','Rank']).sort_index()

TICKER        PRC
date       Rank                  
2013-06-30 T1     MSFT   34.54500
           T1       FB   24.88000
           T2      IBM  191.11000
           T2     AMZN  277.69000
           T3     AAPL  396.53000
           T3    GOOGL  880.37000
2014-06-30 T1     MSFT   41.70000
           T1       FB   67.29000
           T2      IBM  181.27000
           T2     AAPL   92.93000
           T3     AMZN  324.78000
           T3    GOOGL  584.66998
2015-06-30 T1     MSFT   44.15000
           T1       FB   85.76500
           T2      IBM  162.66000
           T2     AAPL  125.42500
           T3     AMZN  434.09000
           T3    GOOGL  540.03998
2016-06-30 T1     MSFT   51.17000
           T1     AAPL   95.60000
           T2      IBM  151.78000
           T2       FB  114.28000
           T3     AMZN  715.62000
           T3    GOOGL  703.53003
2017-06-30 T1     MSFT   68.93000
           T1     AAPL  144.02000
           T2      IBM  153.83000
           T2       FB  150.98000
           T3     AMZN  968.00000
           T3    GOOGL  929.67999

In [12]:
prices_terciles.drop('PRC',axis=1,inplace=True)

In [13]:
prices_terciles.index=prices_terciles.index+pd.offsets.MonthEnd(1)
prices_terciles.head()

,TICKER,Rank
date,,
2013-07-31,MSFT,T1
2014-07-31,MSFT,T1
2015-07-31,MSFT,T1
2016-07-31,MSFT,T1
2017-07-31,MSFT,T1


In [14]:
data.head(10)

,TICKER,PRC,RET
date,,,
2013-01-31,MSFT,27.450,0.027717
2013-02-28,MSFT,27.800,0.021129
2013-03-31,MSFT,28.605,0.028957
2013-04-30,MSFT,33.100,0.157140
2013-05-31,MSFT,34.900,0.061329
2013-06-30,MSFT,34.545,-0.010172
2013-07-31,MSFT,31.840,-0.078304
2013-08-31,MSFT,33.400,0.056219
2013-09-30,MSFT,33.280,-0.003593


In [15]:
new_df=pd.merge_asof(data.drop('PRC',axis=1).sort_index(),prices_terciles.sort_index(),by='TICKER',on='date').dropna()
new_df.set_index(['date','Rank']).sort_index().head(20)

TICKER       RET
date       Rank                 
2013-07-31 T1     MSFT -0.078304
           T1       FB  0.479100
           T2      IBM  0.020564
           T2     AMZN  0.084735
           T3     AAPL  0.141225
           T3    GOOGL  0.008383
2013-08-31 T1     MSFT  0.056219
           T1       FB  0.122120
           T2      IBM -0.060603
           T2     AMZN -0.067193
           T3     AAPL  0.083389
           T3    GOOGL -0.046015
2013-09-30 T1     MSFT -0.003593
           T1       FB  0.216400
           T2      IBM  0.015965
           T2     AMZN  0.112677
           T3     AAPL -0.021481
           T3    GOOGL  0.034254
2013-10-31 T1     MSFT  0.063852
           T1       FB -0.000498

In [99]:
new_df.drop('TICKER',axis=1).groupby(['date','Rank']).mean().unstack(1)

RET                    
Rank              T1        T2        T3
date                                    
2013-07-31  0.200398  0.052650  0.074804
2013-08-31  0.089169 -0.063898  0.018687
2013-09-30  0.106403  0.064321  0.006386
2013-10-31  0.031677  0.066068  0.136484
2013-11-30  0.010618  0.044604  0.048911
2013-12-31  0.071807  0.028523  0.033293
2014-01-31  0.078218 -0.079306 -0.026964
2014-02-28  0.056978  0.031468  0.043339
2014-03-31 -0.025057 -0.015770 -0.031622
2014-04-30 -0.011015 -0.037579  0.028878
2014-05-31  0.039590 -0.014177  0.073519
2014-06-30  0.040799  0.011184  0.025219
2014-07-31  0.057334  0.043052 -0.022530
2014-08-31  0.044476  0.043059  0.044040
2014-09-30  0.038436 -0.014959 -0.019286
2014-10-31 -0.019257 -0.031001 -0.043785
2014-11-30  0.030529  0.049341  0.037767
2014-12-31 -0.012164 -0.041279 -0.058543
2015-01-31 -0.078646  0.008492  0.077670
2015-02-28  0.066690  0.081963  0.059474
2015-03-31 -0.015885 -0.020132 -0.017648
2015-04-30  0.077254  0.036506  0.061411
2015-05-31 -0.012445  0.021580  0.005688
2015-06-30  0.012598 -0.039234  0.000821
2015-07-31  0.076947 -0.018503  0.226307
2015-08-31 -0.055087 -0.072567 -0.029052
2015-09-30  0.011130 -0.020780 -0.008268
2015-10-31  0.161798  0.024839  0.188919
2015-11-30  0.030793 -0.000560  0.048338
2015-12-31  0.012410 -0.061570  0.018277
2016-01-31  0.032554 -0.084235 -0.076465
2016-02-29 -0.058515  0.029543 -0.058351
2016-03-31  0.076324  0.141526  0.069056
2016-04-30 -0.033274 -0.088151  0.019487
2016-05-31  0.040222  0.067204  0.076847
2016-06-30 -0.036328 -0.027704 -0.035220
2016-07-31  0.098872  0.071386  0.092583
2016-08-31  0.021859  0.007736  0.005879
2016-09-30  0.033970  0.008429  0.053297
2016-10-31  0.022306 -0.005639 -0.024727
2016-11-30 -0.004698 -0.015676 -0.045850
2016-12-31  0.039577 -0.002609  0.010212
2017-01-31  0.044069  0.092057  0.066585
2017-02-28  0.064625  0.039223  0.028173
2017-03-31  0.039037  0.008222  0.026253
2017-04-30  0.019704 -0.010904  0.066932
2017-05-31  0.046829 -0.015189  0.071477
2017-06-30 -0.035122  0.002346 -0.042463
2017-07-31  0.043699  0.030732  0.018725
2017-08-31  0.070372  0.007551  0.001516
2017-09-30 -0.031995  0.003968 -0.000143
2017-10-31  0.106734  0.057841  0.105319
2017-11-30  0.018651 -0.003421  0.033846
2017-12-31  0.000516 -0.003818  0.005221